# Project Recommender System

<table align="left">
    <tr>
        <td style="text-align:left">Kode Mata Kuliah</td>
        <td style="text-align:left">:</td>
        <td style="text-align:left">12S3205</td>
    </tr>
    <tr>
        <td style="text-align:left">Nama Mata Kuliah</td>
        <td style="text-align:left">:</td>
        <td style="text-align:left">Sistem Rekomendasi</td>
    </tr>
    <tr>
        <td style="text-align:left">Topik</td>
        <td style="text-align:left">:</td>
        <td style="text-align:left"><i>Sistem Rekomendasi untuk Pariwisata Toba</i></td>
    </tr>
    <tr>
        <td style="text-align:left">Mahasiswa 1</td>
        <td style="text-align:left">:</td>
        <td style="text-align:left"><i>12S21012 - Walker Valentinus Simanjuntak</i></td>
    </tr>
    <tr>
        <td style="text-align:left">Mahasiswa 2</td>
        <td style="text-align:left">:</td>
        <td style="text-align:left"><i>12S21021 - Naomi Elena Lumbanraja</i></td>
    </tr>
    <tr>
        <td style="text-align:left">Mahasiswa 3</td>
        <td style="text-align:left">:</td>
        <td style="text-align:left"><i>12S21033 - Sry Deviani Tambunan</i></td>
    </tr>
</table>

In [1]:
## Data Pre-processing

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("data\Tempat-Wisata-Toba-Preprocessing.csv")

In [4]:
data.shape

(43226, 9)

In [5]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'address', 'PlaceID',
       'Nama_tempat_wisata', 'Category', 'ReviewerId', 'Rating', 'Reviews'],
      dtype='object')

In [6]:
data = data.rename(columns={'ReviewerId': 'user_id', 'PlaceID': 'item_id', 'Rating': 'rating'})
data['user_id'] = data['user_id'].astype(str)
data['item_id'] = data['item_id'].astype(str)
data['rating'] = data['rating'].astype(float)
data.dropna(inplace=True)
data.head(5)

,Unnamed: 0.1,Unnamed: 0,address,item_id,Nama_tempat_wisata,Category,user_id,rating,Reviews
1,1,1,"Jl. Sibola Hotang, Sibola Hotangsas, Kec. Bali...",0,PASIR PUTIH LUMBAN BULBUL,Wisata Bahari,1.11909e+20,5.0,bagus
3,3,3,"Jl. Sibola Hotang, Sibola Hotangsas, Kec. Bali...",0,PASIR PUTIH LUMBAN BULBUL,Wisata Bahari,1.13072e+20,5.0,sangat menyenagkan
4,4,4,"Jl. Sibola Hotang, Sibola Hotangsas, Kec. Bali...",0,PASIR PUTIH LUMBAN BULBUL,Wisata Bahari,1.06173e+20,5.0,bebas foto dimana aja cuma 2k
6,6,6,"Jl. Sibola Hotang, Sibola Hotangsas, Kec. Bali...",0,PASIR PUTIH LUMBAN BULBUL,Wisata Bahari,1.14239e+20,5.0,amazing pengen kesini lagi
8,8,8,"Jl. Sibola Hotang, Sibola Hotangsas, Kec. Bali...",0,PASIR PUTIH LUMBAN BULBUL,Wisata Bahari,1.04743e+20,3.0,jalan menuju lokasi perlu diperhatikan oleh pe...


In [7]:
data.shape

(22166, 9)

In [8]:
# Step 1: Split into 85% training+validation and 15% test
train_val_data, test_data = train_test_split(data, test_size=0.15, random_state=42)
# Step 2: Define validation size as 17% of train+validation, rounded up
val_size = int(len(train_val_data) * 0.17) + 1
# Step 3: Split 85% data into training (about 70% of original) and validation (about 15% of original)
train_data, val_data = train_test_split(train_val_data, test_size=val_size, random_state=42)
# Check sizes
print(f"Train data size: {train_data.shape}")
print(f"Validation data size: {val_data.shape}")
print(f"Test data size: {test_data.shape}")

Train data size: (15638, 9)
Validation data size: (3203, 9)
Test data size: (3325, 9)


Penjelasan Langkah-Langkah

1. Membagi Data Menjadi Training+Validation dan Test:
   - train_test_split(data, test_size=0.15, random_state=42): Fungsi ini digunakan untuk membagi dataset awal menjadi dua bagian, yaitu:
     - 85% untuk Training+Validation (train_val_data)
     - 15% untuk Test (test_data)
   - Pembagian ini bertujuan untuk menyisihkan sebagian data sebagai data uji akhir (test data), yang akan digunakan untuk mengevaluasi kinerja model setelah tahap pelatihan dan tuning selesai.

2. Menentukan Ukuran Data Validation:
   - val_size = int(len(train_val_data) * 0.17) + 1:
     - Setelah membagi data awal menjadi training+validation dan test, kita menentukan bahwa 17% dari train_val_data akan digunakan sebagai data validasi.
     - Penambahan +1 memastikan bahwa ukuran validasi selalu dibulatkan ke atas, agar kita mendapatkan sekitar 17% dari
   - train_val_data sebagai data validasi.

3. Membagi Training+Validation Menjadi Training dan Validation:
   - train_test_split(train_val_data, test_size=val_size, random_state=42): Fungsi ini membagi train_val_data menjadi dua bagian:
     - Training Data (train_data), sekitar 70% dari data asli.
     - Validation Data (val_data), sekitar 15% dari data asli.
   - Pembagian ini bertujuan untuk menyediakan data validasi yang akan digunakan untuk tuning model, misalnya dalam memilih hyperparameter.

4. Menampilkan Ukuran Setiap Subset:
   - print(f"Train data size: {train_data.shape}"): Menampilkan ukuran data training.
   - print(f"Validation data size: {val_data.shape}"): Menampilkan ukuran data validasi.
   - print(f"Test data size: {test_data.shape}"): Menampilkan ukuran data uji.ilkan ukuran data uji.

Analisis Pembagian Data
1. Pembagian data yang dilakukan di sini cukup standar dalam machine learning dan sangat penting untuk memastikan bahwa model tidak overfitting atau underfitting.
2. Training Set (70%) digunakan untuk melatih model, mempelajari pola-pola dalam data, dan mengoptimalkan parameter model.
3. Validation Set (15%) menyediakan data yang terpisah dari training untuk tuning hyperparameter dan memilih model terbaik. Ini mencegah model dari overfitting pada training data karena kita melakukan evaluasi selama proses tuning pada data yang belum pernah dilihat oleh model.
4. Test Set (15%) adalah bagian dari data yang sepenuhnya terpisah, digunakan untuk mengevaluasi performa akhir model setelah proses pelatihan dan tuning selesai. Ini memberikan gambaran yang lebih objektif tentang bagaimana model akan berkinerja pada data baru atau data di dunia nyata.

In [9]:
## Model Pembanding CBF - (BERT)

In [11]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/991.5 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/991.5 kB ? eta -:--:--
   -------------------- ----------------- 524.3/991.5 kB 989.2 kB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\ProgramData\anaconda3\python.exe -m pip install --upgrade pip


In [12]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

ImportError: cannot import name 'BertModel' from 'transformers' (C:\ProgramData\anaconda3\Lib\site-packages\transformers\__init__.py)

In [ ]:
# Step 1: Load and preprocess dataset
data = pd.read_csv("Tempat-Wisata-Toba-Preprocessing.csv")
import pandas as pd

import pandas as pd

# Path to the dataset
file_path_new = 'Tempat-Wisata-Toba-Preprocessing.csv'

# Load the dataset
dataset_new = pd.read_csv(file_path_new)

# Display the first few rows to verify
dataset_new.head()


In [ ]:
# Combine 'Nama_tempat_wisata' and 'Reviews' to form the content description
dataset_new['description'] = dataset_new['Nama_tempat_wisata'] + " " + dataset_new['Reviews'].fillna('')

# Display the updated dataset
dataset_new.head()


In [ ]:
# Drop duplicates based on 'PlaceID'
dataset_new = dataset_new.drop_duplicates(subset='PlaceID')

# Display the dataset to verify changes
dataset_new.head()


In [ ]:
# Step 2: Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')


In [ ]:
# Step 3: Function to get BERT embeddings for text
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)  # CLS token
    return embedding.numpy()# Step 4: Compute embeddings for each place
dataset_new['embedding'] = dataset_new['description'].apply(get_bert_embedding)

# Display the updated dataset
dataset_new.head()


In [ ]:
# Step 4: Compute embeddings for each place
dataset_new['embedding'] = dataset_new['description'].apply(get_bert_embedding)

# Display the updated dataset
dataset_new.head()


In [ ]:
# Step 5: Build similarity matrix
embeddings = np.array(dataset_new['embedding'].tolist())
similarity_matrix = cosine_similarity(embeddings)

# Verify the similarity matrix
similarity_matrix.shape


In [ ]:
# Step 6: Recommendation function
def recommend_places(place_id, dataset, similarity_matrix, top_n=5):
    try:
        place_index = dataset[dataset['PlaceID'] == place_id].index[0]
        similarity_scores = similarity_matrix[place_index]
        top_indices = similarity_scores.argsort()[-top_n-1:-1][::-1]  # Exclude itself
        recommendations = dataset.iloc[top_indices]
        return recommendations[['PlaceID', 'Nama_tempat_wisata', 'Category']]
    except IndexError:
        return "Place ID not found in the dataset."

In [ ]:
# Example: Get recommendations for a specific place
place_id = 0  # Replace with a valid PlaceID from your dataset
recommendations = recommend_places(place_id, dataset, similarity_matrix, top_n=5)
print(f"Top recommendations for PlaceID {place_id}:\n", recommendations)